# A Practical Guide to Computing the Diffusivity of Amorphous Materials Using VASP

This guide provides a detailed procedure—along with all necessary scripts—to compute the diffusivity of any amorphous material at room temperature using the VASP package. 
For scientific background, see [Angew. Chem. Int. Ed. 2024, 63, e202315628](https://onlinelibrary.wiley.com/doi/full/10.1002/anie.202315628), and [ACS Appl. Mater. Interfaces 2020, 12, 35748](https://pubs.acs.org/doi/10.1021/acsami.0c10000). We will use the material studied in [Angew. Chem. Int. Ed. 2024, 63, e202315628](https://onlinelibrary.wiley.com/doi/full/10.1002/anie.202315628), combined with methods from [ACS Appl. Mater. Interfaces 2020, 12, 35748](https://pubs.acs.org/doi/10.1021/acsami.0c10000) to achieve efficient and accurate diffusivity calculations.

# 0. Preparation

First of all, the user needs to have the composition, approximate density and approximate melting point of the material. 

The user also needs to install some packages. 

The calculations relies on the VASP package. The user can provide the location of POTCAR directory to generate the POTCAR file used throughout the procedure.

Download and compile [packmol](https://m3g.github.io/packmol/). The location of the executable must be provided to generate the initial amorphous structure.

The code was tested in a Conda environment, which is the recommended way to run this workflow. Assuming you have Conda installed or loaded, use the following command to create an environment for this project:

<code>conda create --name vaspenv python numpy scipy ase pymatgen</code>. 

Then type <code>conda activate vaspenv</code> to activate the environment.

The procedure is illustrated below: ![procedure](attachement:procedure.png)

The <code>POSCAR</code> is generated in section 1 and then handled automatically by the code. 

The same <code>POTCAR</code> and <code>KPOINTS</code> files will be used throughout the project. 

The <code>POTCAR</code> is automatically generated in section 1, provided that the user supply the location of POTCAR files. The default is the [PBE pseudopotentials recommended by VASP](https://www.vasp.at/wiki/index.php/Available_pseudopotentials) stored in <code>utils.py</code>. The user is also allowed to provide a customized <code>potcarDict</code> in section 1 to override it.

We use a gamma point in <code>KPOINTS</code> since we are simulating amorphous materials with a large box.

The <code>INCAR_NVT</code> template is used throughout the initial equilibration, scaling of lattice, and quench.

The <code>INCAR_NPT_MLFF</code> template is used in machine learning force field training, validation, and running. 

The <code>INCAR_OPT</code>  is used in optimization of atomic positions at 0 K.

Some default parameters in INCAR files are:

<code>ENCUT = 1.3 * ENMAX</code>

<code>POTIM = 1.0</code> if the lightest atom in the system is H or Li, else <code>POTIM = 2.0</code>.

The user needs to supply slurm scripts in section 1 to run VASP. The user can provide the same scripts for AIMD and MLFF simulations. However, if GPU is available, it preferable to run AIMD on GPUs and pure MLFF on CPUs.


# 1. Equilibrate and Optimize the Density in the Liquid Phase
The first step is to melt and equilibrate the material above its melting point but below its boiling point. We follow [ACS Appl. Mater. Interfaces 2020, 12, 35748](https://pubs.acs.org/doi/10.1021/acsami.0c10000) to create a randomly packed initial structure using the packmol package.

<code>cd initialStructure</code> 

Modify the user input section of <code>getInitialStructure.py</code> for your own system.

```python
# ----- user data -------------------------------------------------------------
molar_masses = {"Li": 6.9410, "Ta": 180.94788, "Cl": 35.45}     # g mol⁻¹
stoich       = {"Li": 1, "Ta": 1, "Cl": 6}                      # LiTaCl₆
density      = 2.96                      # g cm⁻³
melting_point = 1500 # K, will be rounded to the nearest multiples of 300 K, guranteed to be equal to or larger than the original value.
box_diam     = 12.0                      # Å
packmolPath = "/home/xiaolin/software/packmol-20.15.2/packmol"
potcarPath = "/home/xiaolin/VASP/paw-pbe-64"
AIMDslurm = "run_kepler_gpu.sh"
MLFFslurm = "run_kepler_cpu.sh"
#potcarDict = {"Li": "Li_sv", "Ta": "Ta_pv", "Cl": "Cl"}  # same as default
# ---------------------------------------------------------------------------
```
The user needs to supply molar masses and stoichiometry for each element in the system, as well as density, diameter of the cubic box, and the location of packmol package. Note the code will create a cubic box with diameters slightly greater than <code>box_diam</code>.
Run the code and you will get a <code>POSCAR</code> file in the same directory. This is the starting geometry for our simulations.

We then equilibrate the structure at a suitable temperature. For $LiTaCl_6$, we choose 1500 K. Copy the POSCAR file to <code>eql_1500</code> and prepare other files. To better reach equilibrium, a Langevin thermostat should be used. Run <code>pltAll.py</code> frequently to monitor the status of MD. Modify the user input section of <code>pltAll.py</code> for each directory you want to monitor:

```python
# ----- user data -------------------------------------------------------------
timestep = 1.0 # in fs
timeAvg = 2 # in ps
maxRestart = 10

last_n_ps_steps = int(timeAvg * 1e3 / timestep)
dirs = ["eql_1500"]
# ---------------------------------------------------------------------------   
```
The code plots potential energy, temperature, pressure and volume in dirs and restart1/ up to restart10/ directories within each of them, and saves a png file named using the directories. A green curve showing the running average property of <code>timeAvg</code> is also plotted. It is considered to have reached equilibrium if the difference of average energy between the last 2 ps and the last 4 ps is within 1 meV/atom.

Once it reached equilibrium, we need to determine its density at 1500 K. We do so by iteratively scale the lattice parameters according to:

$\frac{\Delta V}{V} \approx -\frac{\Delta P}{K}$, 

where $V$ is the volume of the cell, $P$ is pressure, and $K$ is bulk modulus. We approximate $K$ by $100GPa$.

Copy the equilibrated <code>CONTCAR</code> to <code>scale/POSCAR</code>. Copy other input files and modify <code>iterativeScale.sh</code> for your own project. The script runs AIMD for 4 ps at most 10 times and takes the average pressure in the last 2 ps as input to scale the lattice until the pressure drops below 5 kB. If you are going to sample structures for better estimates of diffusivities and activation energies, run a longer simulation based on the resulting <code>CONTCAR</code>.

# 2. Quench and Find the Density at Room Temperature
Having obtained a good liquid structure, we then quench the material down to room temperature. 

Copy the <code>POSCAR</code> from <code>scale/</code> to <code>quench/</code>

The <code>quench/quench.sh</code> script quenches the material from 1500 K to 300 K in a decrement of 300 K every 2 ps. Modify it for your own project.


Having quenched down to 300 K, we then quench the material down to 0 K to optimize for more accurate density at room temperature.

Since the density is usually larger at room temperature, we optimize the atomic positions at scaled lattices from 0.98 to 1.0 in an increment of 0.001. 

<code>cd opt</code>

And run <code>scaledOPT.py</code>. The code assumes <code>../quench/quench_300/CONTCAR</code> exists.
After all optimization jobs converged, run <code>pltEScaled.py</code> to plot the results and find the scaling factor corresponding to the lowest energy. We will use the resulting <code>CONTCAR</code> for training and running machine learning force field (MLFF).

# 3. Train the Machine Learning Force Field
We are going to run long NVT simulations at 300 K up to 500 K in an increment of 50 K. To obtain reliable diffusivities, we need very long MD trajectories. So we train MLFF by heating from 300 K to 600 K during 60 ps in chunks of 20 ps using an NPT ensemble with Langevin thermostat. This is to avoid large deformations of the cell in a single run.

# 4. Refit for Fast Production Run
To run MLFF at ns/day, we need to refit the force field. This part is memory intensive, but can be fast if enough memory is supplied. On perlmutter, we can use 4 CPU nodes on the debug queue. 

# 5. Validate the Force Field 
By doing the refit, we can obtain the training errors. We also need test errors to see how well the MLFF generalizes. 

# 6. Run the MLFF to Determine Diffusivities and Activation Energy